In [1]:
import argparse
import pathlib
import random
from collections import deque
from typing import List, Dict, Tuple, Union
from data import Dataset
from data_utils import *
from sampling import *
from models import BetaBernoulli
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
%matplotlib inline  

import matplotlib;matplotlib.rcParams['font.size'] = 10
import matplotlib;matplotlib.rcParams['font.family'] = 'serif'

LOG_FREQ = 10
output = pathlib.Path("../output/active_learning_low_variance")

In [2]:
dataset_name = 'cifar100'
metric = 'accuracy' # 'ece', 'confusion_matrix'
group_method = 'predicted_class'
pseudocount = 2
RUNS = 100
experiment_name = '%s_%s_groupby_%s_runs%d_pseudocount%.2f' % \
            (dataset_name, metric, group_method, RUNS, pseudocount)

In [ ]:
samples = {}
mpe_log = {}
method_list = ['random_arm', 'random_data', 'ts_uniform', 'ts_informed']
for method in method_list:
    samples[method] = np.load(open(output / experiment_name / ('samples_%s.npy' % method), 'rb'))
    mpe_log[method] = np.load(open(output / experiment_name / ('mpe_log_%s.npy' % method), 'rb'))

In [ ]:
method_list = ['random_arm', 'random_data', 'ts_uniform', 'ts_informed']
method_format = {'random_arm': ('Random Arm', 'b', '.', '-.'), 
                 'random_data': ('Random Datapoint', 'g', '^', '-.'), 
                 'ts_uniform': ('TS Uniform', 'r', '*', '-'), 
                 'ts_informed': ('TS informed', 'r', '+', '-.'),
                }

dataset = Dataset.load_from_text(dataset_name)
dataset.group(group_method = group_method)
start = 3
num_steps = 1000
stepsize = 1

plt.figure(figsize=(6,4))
for method_name in method_list:
    legend_name, color, marker, linestyle = method_format[method_name]
    estimation_error = np.zeros((RUNS, dataset.__len__() // LOG_FREQ))
    for run_id in range(RUNS):
        accuracy_k = dataset.accuracy_k
        accuracy_k[np.isnan(accuracy_k)] = 0
        estimation_error[run_id] = np.sqrt(np.inner(np.abs(mpe_log[method_name][run_id] - accuracy_k)**2, \
                                                    dataset.weight_k))
    
    xrange = np.arange(num_steps) * LOG_FREQ
    mean_error = np.mean(estimation_error, axis=0)
    lb_error = np.quantile(estimation_error, 0.125, axis=0)
    ub_error = np.quantile(estimation_error, 0.875, axis=0)

    idx = np.arange(num_steps)[start:(start+num_steps)][::stepsize]   
    plt.plot(xrange[idx], 
             mean_error[idx], 
             label=legend_name, linewidth=3, color=color, linestyle=linestyle)
    plt.fill_between(xrange[idx], 
                     lb_error[idx], 
                     ub_error[idx], 
                     color=color, alpha=.2)
    #plt.ylim([0, 0.3])
    plt.xlabel('#Labeled')
    plt.ylabel('$L_2$ Error')
    plt.xscale('log')
    plt.legend(fontsize=12, loc='upper right')
    plt.title(experiment_name)

In [ ]:
run_id = 1
dataset = Dataset.load_from_text(dataset_name)
dataset.group(group_method = group_method)
fig, axes = plt.subplots(1, len(method_list), figsize=(12,3), sharex=True, sharey=True)
for i, method_name in enumerate(method_list):
    legend_name, color, marker, linestyle = method_format[method_name]
    for N in [200, 50]:
        indices = samples[method_name][run_id][:N]
        categories = dataset.categories[indices]
        axes[i].hist(categories, label=N, alpha=0.8, bins = 100)
    axes[i].legend()
    axes[i].set_xlabel('Predicted class')
    axes[i].set_ylabel('#labeled')
    axes[i].set_title(legend_name)